In [7]:
import Modules.SQcircuit_extensions as sq_ext
import Modules.figures as figs
import SQcircuit as sq
import numpy as np
import matplotlib.pyplot as plt
import importlib
from scipy.optimize import minimize_scalar
from scipy.linalg import eigh
import qutip as qt
import sympy as sym
from joblib import Parallel, delayed, cpu_count

%matplotlib ipympl
plt.rcParams.update({
    "text.usetex": True,
    "font.family": "serif",
    "font.serif": ["Palatino"],
})
# plt.rcParams['text.usetex'] = False
importlib.reload(sq_ext)
importlib.reload(sq)
importlib.reload(figs)
np.set_printoptions(linewidth=300, formatter={'float': '{:.12f}'.format})

In [8]:
fF  = 1e-15
nH  = 1e-9
GHz = 1e9

In [30]:
CR_1, CF_1, LF_1, LR_1, EJ_1, Δ_1, ω_r_1 = sq_ext.get_experimental_parameters('qubit_A', Δ=0.1, LR=100)
CR_2, CF_2, LF_2, LR_2, EJ_2, Δ_2, ω_r_2 = sq_ext.get_experimental_parameters('qubit_B', Δ=0.1, LR=140)

In [31]:
input_circuit = sq_ext.sq_fluxonium(C_F_eff=CF_2, L_F_eff=LF_2, Δ=Δ_2, EJ=EJ_2)
sq_ext.diag(input_circuit.hamiltonian(), n_eig=2, remove_ground=True)[0][1]


3.0342472605322754

In [32]:
H_target = sq_ext.sq_fluxonium(C_F_eff=CF_1, L_F_eff=LF_1, Δ=Δ_1, EJ=EJ_1).hamiltonian()

In [33]:
ω_target = sq_ext.diag(H_target, n_eig=2, remove_ground=True)[0][1]
ω_target

3.700194489064325

In [34]:
def find_resonance(H_target, input_circuit):
    # Step 1: Calculate the target gap ω_target
    ω_target = sq_ext.diag(H_target, n_eig=2, remove_ground=True)[0][1]
    
    # Step 2: Define the objective function to minimize the difference between ω_target and ω_input
    def objective(φ_ext):
        # Set the external flux of the input circuit
        loop = input_circuit.loops[0]
        loop.set_flux(φ_ext)
        
        # Diagonalize the input circuit Hamiltonian
        E_input = input_circuit.diag(n_eig=2)[0]
        ω_input = E_input[1] - E_input[0]
        
        # Return the absolute difference between ω_target and ω_input
        return np.abs(ω_target - ω_input)
    
    # Step 3: Use an optimization method to find the optimal φ_ext
    result = minimize_scalar(objective, bounds=(0.5, 1), method='bounded')
    
    # Return the optimal φ_ext and the corresponding gap
    optimal_φ_ext = result.x
    optimal_gap = ω_target
    
    return optimal_φ_ext, optimal_gap

In [35]:
find_resonance(H_target, input_circuit)

(0.5500060147953606, 3.7001944890643346)